### Επεξεργασία κλινικών δεδομένων για καρδιαγγειακή πάθηση

* Χρησιμοποιείται το σύνολο δεδομένων "Heart Disease" από το UCI repository (με ID = 45) το οποίο μπορείτε να δείτε εδώ: https://archive.ics.uci.edu/dataset/45/heart+disease

* Το σύνολο δεδομένων αποτελείται από 303 δείγματα/γραμμές (ασθενείς) και 14 χαρακτηριστικά (συμπεριλαμβανομένης και της μεταβλητής εξόδου).

In [1]:
# Θα πρέπει να χρησιμοποιηθεί 
from ucimlrepo import fetch_ucirepo 
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

* Συνάρτηση μέσω της οποίας έχουμε την δυνατότητα να μετατρέψουμε οποιαδήποτε αριθμητική μεταβλητή σε κατηγορική (με βάση ένα threshold).

In [2]:
def convert_to_categorical(df, column_name, threshold):
    """
    Convert numeric values in a specified column of a dataframe
    to 'health' if they are above a threshold or 'disease' otherwise.
    
    Parameters:
    - df: DataFrame containing the data
    - column_name: string, the name of the column to convert
    - threshold: numeric, the threshold value
    
    Returns:
    - DataFrame with the specified column converted
    """
    if column_name not in df.columns:
        raise ValueError(f"The column '{column_name}' is not in the dataframe.")

    # Directly apply the conversion to the column within the dataframe
    df[column_name] = df[column_name].apply(lambda x: 'normal' if x > threshold else 'risk')
    
    return df

* Με την συνάρτηση "fetch_ucirepo" έχουμε την δυνατότητα να "τραβήξουμε" οποιοδήποτε σύνολο δεδομένων θέλουμε από το UCI repository (χρησιμοποιώντας ως όρισμα το id του)

In [3]:
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 

* Έχουμε την δυνατότητα να απομονώσουμε τον πίνακα με τα χαρακτηριστικά (dataframe) καθώς επίσης και την μεταβλητή εξόδου (tag) χρησιμοποιώντας τις μεθόδους ".data.features" και ".data.targets"

In [4]:
# data (as pandas dataframes) 
df = heart_disease.data.features 
tag = heart_disease.data.targets 

In [5]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0
301,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0


* Μπορούμε εύκολα μέσω της εντολής type να επιβεβαιώσουμε ότι το dataset που έχουμε στην διάθεσή μας είναι σε μορφή pandas dataframe

In [6]:
type(df)

pandas.core.frame.DataFrame

* Μπορούμε να δούμε τις τιμές που παίρνει η μεταβλητή εξόδου.

In [7]:
tag["num"].value_counts()

0    164
1     55
2     36
3     35
4     13
Name: num, dtype: int64

* Παρατηρούμε ότι η μεταβλητή εξόδου παίρνει 5 τιμές, από 0 έως 4. Σύμφωνα με την περιγραφή του dataset στο UCI repository, η τιμή 0 αντιστοιχεί σε απουσία καρδιαγγειακού νοσήματος "0 (no presence)" ενώ η παρουσία καρδιαγγειακού νοσήματος έχει κωδικοποιηθεί με τις τιμές 1, 2, 3 και 4 (" presence (values 1,2,3,4)").

* Εφόσον έχουμε 2 μεγάλες κατηγορίες (απουσία vs παρουσία καρδιαγγειακού νοσήματος), μπορούμε να απλοποιήσουμε την μεταβλητή εξόδου έτσι ώστε το πρόβλημα να γίνει δυαδικής ταξινόμησης (binary classification).

* Για να το πετύχουμε αυτό, θα πρέπει πρώτα να δημιουργήσουμε ένα dictionary στο οποίο κάθε μια από τις "παλιές" τιμές που φέρει η μεταβλητή εξόδου θα αντιστοιχεί σε μια "καινούργια" (0 ή 1).

In [8]:
TAG_VALUES = {
    0: 0,
    1: 1,
    2: 1,
    3: 1,
    4: 1,
}

tag['num'] = tag['num'].replace(TAG_VALUES)

In [9]:
tag["num"].value_counts()

0    164
1    139
Name: num, dtype: int64

* Από την περιγραφή του dataset στο UCI repository παρατηρούμε ότι έχει και missing values.

In [10]:
# Check for any missing values in each column
missing_values = df.isna().any()
missing_values[missing_values == True].index.tolist()

['ca', 'thal']

* Παρατηρούμε ότι έχουμε missing values σε δύο στήλες/χαρακτηριστικά του dataset.

* Μπορούμε να δούμε σε ποιές γραμμές έχουμε missing values.

In [11]:
# Find the rows with missing values
missing_values_rows = df[df.isna().any(axis=1)]

# Display the rows with missing values
missing_values_rows

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
87,53,0,3,128,216,0,2,115,0,0.0,1,0.0,NaN
166,52,1,3,138,223,0,0,169,0,0.0,1,NaN,3.0
192,43,1,4,132,247,1,2,143,1,0.1,2,NaN,7.0
266,52,1,4,128,204,1,0,156,1,1.0,2,0.0,NaN
287,58,1,2,125,220,0,0,144,0,0.4,2,NaN,7.0
302,38,1,3,138,175,0,0,173,0,0.0,1,NaN,3.0


* Σε περίπτωση που έχουμε καθοδήγηση από κάποιον ειδικό, μπορούμε να αντικαταστήσουμε τα missing values με συγκεκριμένες τιμές.

* Εναλλακτικά μπορούμε να αφαιρέσουμε είτε τα χαρακτηριστικά/στήλες είτε τα δείγματα/γραμμές όπου συναντούμε τα missing values.

* Πριν αφαιρέσουμε όμως τις γραμμές που φέρουν NaN τιμές, προσθέτουμε μια στήλη στο dataframe η οποία θα φέρει τις τιμές της μεταβλητής εξόδου, έτσι ώστε η αλλαγή που θα γίνει στις γραμμές του dataset να συμβαδίζει και με τις τιμές στο tag.

In [12]:
df["tag"] = tag["num"]
df = df.dropna()
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,tag
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,4,140,241,0,0,123,1,0.2,2,0.0,7.0,1
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,1
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,1


In [13]:
# Εναλλακτικά μπορούμε να αφαιρέσουμε γραμμές με βάση τις NaN τιμές συγκεκριμένων στηλών/χαρακτηριστικών
# df = df.dropna(subset=['ca', 'thal'])

* Μπορούμε να αποθηκεύσουμε το dataset σε ένα αρχείο .csv το οποίο θα χρησιμοποιήσουμε στην συνέχεια για περαιτέρω επεξεργασία και ανάλυση.

In [ ]:
df.to_csv("./data/uci_heart_failure/uci_hf_df.csv", index=False)

* Με την παραπάνω συνάρτηση έχουμε την δυνατότητα να μετατρέψουμε αριθμητικές τιμές του dataset σε κατηγορικές

In [14]:
df = convert_to_categorical(df, 'chol', 260)

In [15]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,tag
0,63,1,1,145,risk,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,normal,0,2,108,1,1.5,2,3.0,3.0,1
2,67,1,4,120,risk,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,risk,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,risk,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,4,140,risk,0,0,123,1,0.2,2,0.0,7.0,1
298,45,1,1,110,normal,0,0,132,0,1.2,2,0.0,7.0,1
299,68,1,4,144,risk,1,0,141,0,3.4,2,2.0,7.0,1
300,57,1,4,130,risk,0,0,115,1,1.2,2,1.0,7.0,1


* Μπορούμε να τοποθετήσουμε τις ηλικίες (ή και άλλες συνεχείς μεταβλητές) σε διακριτά bins (ηλικιακές ομάδες).

* Το binning έχει τα εξής χαρακτηριστικά.

* Πλεονεκτήματα:
    - Μειώνει τον θόρυβο (σε περιπτώσεις όπου υπάρχουν λάθη όσον αφορά στις τιμές του dataset).
    - Συμβάλλει στην ερμηνευσιμότητα των δεδομένων (π.χ: τοποθετώντας ηλικίες σε συγκεκριμένες κατηγορίες).
    - Μειώνει τις μοναδικές τιμές που υπάρχουν στο dataset (απλοποίηση μοντέλου).
    
* Μειονεκτήματα:

    - Χάνεται πληροφορία που αφορά στην διακύμανση των τιμών κάποιων χαρακτηριστικών.
    - Τα bins που δημιουργούνται είναι κάπως αυθαίρετα.
    - Παρατηρήσεις που έχουν τιμές πολύ κοντά στα όρια των bins μπορεί να τοποθετούνται ανεξαιρέτως εντός ενός συγκεκριμένου bin. 

In [16]:
df["age"].unique()

array([63, 67, 37, 41, 56, 62, 57, 53, 44, 52, 48, 54, 49, 64, 58, 60, 50,
       66, 43, 40, 69, 59, 42, 55, 61, 65, 71, 51, 46, 45, 39, 68, 47, 34,
       35, 29, 70, 77, 38, 74, 76], dtype=int64)

In [17]:
bins = [0, 30, 60, 80]

labels = ['Young', 'Middle-Aged', 'Senior']

df['age'] = pd.cut(df['age'], bins=bins, labels=labels, include_lowest=True)

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,tag
0,Senior,1,1,145,risk,1,2,150,0,2.3,3,0.0,6.0,0
1,Senior,1,4,160,normal,0,2,108,1,1.5,2,3.0,3.0,1
2,Senior,1,4,120,risk,0,2,129,1,2.6,2,2.0,7.0,1
3,Middle-Aged,1,3,130,risk,0,0,187,0,3.5,3,0.0,3.0,0
4,Middle-Aged,0,2,130,risk,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Middle-Aged,0,4,140,risk,0,0,123,1,0.2,2,0.0,7.0,1
298,Middle-Aged,1,1,110,normal,0,0,132,0,1.2,2,0.0,7.0,1
299,Senior,1,4,144,risk,1,0,141,0,3.4,2,2.0,7.0,1
300,Middle-Aged,1,4,130,risk,0,0,115,1,1.2,2,1.0,7.0,1


In [18]:
AGE_VALS_NUM = {
    "Young": 0,
    "Middle-Aged": 1,
    "Senior": 2,
}


CHOL_VALS_NUM = {
    "risk": 1,
    "normal": 0,
}

In [19]:
df["age"] = df["age"].replace(AGE_VALS_NUM)
df["chol"] = df["chol"].replace(CHOL_VALS_NUM)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,tag
0,2,1,1,145,1,1,2,150,0,2.3,3,0.0,6.0,0
1,2,1,4,160,0,0,2,108,1,1.5,2,3.0,3.0,1
2,2,1,4,120,1,0,2,129,1,2.6,2,2.0,7.0,1
3,1,1,3,130,1,0,0,187,0,3.5,3,0.0,3.0,0
4,1,0,2,130,1,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,1,0,4,140,1,0,0,123,1,0.2,2,0.0,7.0,1
298,1,1,1,110,0,0,0,132,0,1.2,2,0.0,7.0,1
299,2,1,4,144,1,1,0,141,0,3.4,2,2.0,7.0,1
300,1,1,4,130,1,0,0,115,1,1.2,2,1.0,7.0,1
